Let's start with a review of the URI (Uniform Resource Identifier) syntax.

## Components of a URI (Uniform Resource Identifier)

A URI is a string of characters used to identify a resource over a network. A typical URI consists of the following components:

### 1. Scheme
The **scheme** defines the protocol or method used to access the resource. Examples include `http`, `https`, `ftp`, etc. It is followed by a colon (`:`).

- Example: `http` in `http://example.com`

### 2. Authority
The **authority** section usually contains the hostname and often the port number. It is prefixed by `//`.

- Example: `example.com:8080` in `http://example.com:8080/path`

### 3. Path
The **path** specifies the resource or the specific page that you want to access. It comes after the authority and is separated by slashes (`/`).

- Example: `/path/to/resource` in `http://example.com/path/to/resource`

### 4. Query
The **query** starts with a question mark (`?`) and is used to pass parameters to the resource. Multiple query parameters are separated by ampersands (`&`).

- Example: `?key1=value1&key2=value2` in `http://example.com/path?key1=value1&key2=value2`

### 5. Fragment
The **fragment** is an optional part that starts with a hash (`#`) and is used to identify a specific part of the resource, like a section within a page.

- Example: `#section1` in `http://example.com/path#section1`

Here's how these components fit together to form a complete URI:


- Complete Example: `http://example.com:8080/path/to/resource?key1=value1&key2=value2#section1`


  ### Retrieving Data from the API

Next we need a way to retrieve the data from the API. To achieve this, we first import Python's `requests` library, which allows us to make HTTP requests. Additionally, we import the `pprint` function from the `pprint` library to make the output more readable. Finally, we define the API endpoint we'll be interacting with, setting `API_ENDPOINT` to `https://testing-api.water.noaa.gov/hefs/`. Here's how you can do it:

```python
# start by importing the requests library
import requests
# import the pprint library to make the output more readable
from pprint import pprint
# define the api-endpoint
# Note: API Endpoint will change to https://api.water.noaa.gov/hefs/ after testing is complete
API_ENDPOINT = "https://testing-api.water.noaa.gov/hefs"
```

**Please change the endpoint to match the server you are interacting with.**

From here  you will only need to set the uri, fetch the data, and print the results.  Here is a template for the code:

```python
# set the uri
uri = "/v1/locations"
# fetch the data
r = requests.get(API_ENDPOINT + uri)
# print the results
pprint(r.json())
```



In [ ]:
# start by importing the requests library
import requests
# import the pprint library to make the output more readable
from pprint import pprint
# define the api-endpoint
API_ENDPOINT = "https://testing-api.water.noaa.gov/hefs"

## The Header Endpoint: `/v1/headers/`

The `/v1/headers/` endpoint is a part of the RESTful API that provides access to various series data. By making HTTP requests to this URI, you can retrieve, update, or delete information related to data series stored in the backend. This endpoint by default returns time series sequentially.



In [ ]:
# create a series request
uri = API_ENDPOINT + "/v1/headers/"

# get the response
response = requests.request("GET", uri, headers=None)
# print the response
print(response.headers)

## Pagination

The `/v1/headers/?limit=10&offset=10` endpoint is a specialized version of the `/v1/headers/` URI, which includes query parameters for pagination. In this case, the `limit` parameter is set to 10, which means the API will return a maximum of 10 records in the response.
The offset parameter is set to 10, which means the API will skip the first 10 records in the response. This is useful for retrieving large amounts of data, as it allows you to retrieve data in chunks.

In [ ]:
# create a series request returning the second tranche of 10 series
uri = API_ENDPOINT + "/v1/headers/?limit=10&offset=10"
# get the response
response = requests.request("GET", uri, headers=None)
# print the response
pprint(response.content)

## Filtering Data with the API

The API endpoint `/v1/headers/?limit=10&offset=10` allows you to retrieve a subset of data series.

It uses query parameters to offer flexibility in filtering results. The `limit` parameter controls the number of records returned, while the `offset` parameter lets you skip a certain number of records from the beginning.

You can also filter results based on specific fields like `location_id`, `parameter_id`, and many more. The API filters on all available fields by appending the field name and value to the URI as query parameters.

For instance, if you're interested in series that have a specific `parameter_id` and `location_id`, the URI would look like this:

/v1/headers/?limit=10&offset=10&parameter_id=QINE&location_id=BLUO2


In [ ]:
# create a series request returning the second tranche of 10 series, with location_id=BLUO2 and parameter_id=QINE filters
uri = API_ENDPOINT + "/v1/headers/?limit=10&offset=10&parameter_id=QINE&location_id=BLUO2"
# get the response
response = requests.request("GET", uri, headers=None)
# print the response
pprint(response.json())

### Filtering Data Based on Multiple Parameters

In some use cases, you might want to retrieve data that matches more than one parameter value. The API allows filtering by multiple values for the same parameter. This is achieved by appending `&` to the parameter name and specifying the different values.

For example, to retrieve data series that belong to either `location_id=BLUO2` or `location_id=ABEC2`, you can format the URI like this:

`/v1/headers/?location_id=BLUO2&location_id=ABEC2`

In [ ]:
# create a series request with location_id=ABEC2 and location_id=BLUO2 filters
uri = API_ENDPOINT + "/v1/headers/?location_id=ABEC2&location_id=BLUO2"
# get the response
response = requests.request("GET", uri)
# print the response
pprint(response.json())

### A Quick Example

In other cases, you might want to retrieve and plot recent ensemble data. The API allows ordering by a certain value of a parameter. This is achieved by adding `ordering=` to the parameter name.

For example, to retrieve the newest data series that belongs to `location_id=BLUO2`, you can format the URI like this:

'/v1/ensembles/?location_id=BLUO2&ordering=-end_date_date&limit=1'

Note: This graph will sometimes be flat if there is no measured change in flow rate.

In [ ]:
# create a series request with location_id=BLUO2 filters
uri = API_ENDPOINT + "/v1/ensembles/?location_id=BLUO2&ordering=-end_date_date&limit=1"
# get the response
response = requests.request("GET", uri)
# print the response
pprint(response.json())

In [ ]:
import matplotlib.pyplot as plt
import datetime

def graph_ensemble_data(response_data):
  """Graphs ensemble data from a DRF response.

  Args:
    response_data: A dictionary containing the DRF response data.
  """

  if 'results' in response_data and response_data['results']:
    times = []
    values = []
    for result in response_data['results']:
      if 'events' in result and result['events']:
        for event in result['events']:
          for value in result['events']:
              datetime_str = f"{value['date']} {value['time']}"
              datetime_obj = datetime.datetime.strptime(datetime_str, "%Y-%m-%d %H:%M:%S")
              times.append(datetime_obj)
              values.append(value['value'])
  plt.plot(times, values, label=f"Value as a function of time")
  plt.xlabel('Time')
  plt.ylabel('Value')
  plt.title('Ensemble Data')
  plt.legend()
  plt.xticks(rotation='vertical')
  plt.show()


#pprint(response.json())
graph_ensemble_data(response.json())


## Summary

In this notebook, we learned how to use the HEFS API to retrieve data from the HEFS data store. We also learned how to use the API to filter results based on specific fields like `location_id`, `parameter_id`, and many more. The API filters on all available fields by appending the field name and value to the URI as query parameters.

These techniques can be used to retrieve, filter, and paginate data from all resources the HEFS data store.  With one exception, the GraphQL interface, we cover that next.